## **Custom Dataset로 Yolov8 재학습하여 객체 감지 예측하기**

**< 진행 절차 >**
1. Data 준비하기 : Custom Dataset Dataset으로 Yolov8 재학습(Fine Tuning)하는 경우에는 Image / Annotation 으로 이루어진 Dataset 준비
    - Roboflow에서 제공하는 Training Dataset을 이용하거나 Labeling Tool 을 이용하여 개발자가 직접 Labeling 시킨 Image / Annotation으로 이루어진 Training Dataset을 구축해야함
    - Custom Dataset 구축시 이미지 데이터와 정답데이터는 확장자를 제외한 파일 이름은 동일해야함
    - Yolov8에서 annotation 파일 즉 정답 파일의 확장자는 반드시 .txt 여야 함         

- 1) Roboflow 에서 Aquarium Dataset (custom data) 다운로드
    - Colab 으로 데이터셋 업로드 : Roboflow(https://public.roboflow.com/)에서 제공하는 Dataset 로드하기
- 2) yaml 파일 설정** (데이터셋 위치 알려주는 config file)
    - 2-1) roboflow 에서 제공되는 data.yaml 파일 확인
    - 2-2) custom data에 대한 yaml 파일 만들기
    Yolov8으로 Custom Data를 학습하기 위해서는 YAML 파일이 반드시 필요. YAML 파일에는 다음 정보를 포함해야함
        - 이미지와 정답이 저장되어 있는 디렉토리 정보
        - 인식하고 싶은 클래스 종류와 대응대는 각각의 이름
        - 형식 : 클래스번호 | x의 center 좌표|y의 center좌표| 너비 |높이  
            - 전체 이미지의 width 와 height 값으로 나눈 비율값임

2. ultralytics 패키지 설치하기
```python
    pip install ultralytics
```
3. 모델 객체 선언하고 학습하기
```python
    from ultralytics import YOLO
    model = YOLO('yolov8n.pt')  #  model = YOLO('yolov8n-seg.pt')
    model.train(data='mydata.yaml', epochs=10)
```
4. 객체 검출하기
```python
results = model.predict(source='/content/test/')
```
5. 결과 확인하기
6. 결과 다운로드하기
7. 학습된 모델 내보내기
8. 웹캠에서 모델 사용하여 객체 검출하기

## **1. 데이터 준비하기(Custom Data 구축)**
- Roboflow에서 제공하는 Training Dataset을 이용

### 1) Roboflow 에서 Aquarium Dataset (custom data) 다운로드
- Public Dataset : https://public.roboflow.com/object-detection/aquarium/2
- Download Dataset > Export - Select Format : Yolov8, show download code 선택 후  continue 버튼 클릭 > Your Download Code - Raw URL 탭에서 주조 복사하기

In [ ]:
# Roboflow 에서 직접 다운로드
# Linux에서 실행하는 경우 wget을 사용하여 다운로드합니다.
# !wget -O Aquarium_Data.zip  https://public.roboflow.com/ds/7o8nkc5tC0?key=TflNDoBe5H

# Windows에서 실행하는 경우 request 라이브러리를 사용하여 다운로드합니다.
import requests

url = "https://public.roboflow.com/ds/7o8nkc5tC0?key=TflNDoBe5H"
output_path = "Aquarium_Data.zip"

response = requests.get(url)
with open(output_path, 'wb') as f:
    f.write(response.content)

print("Download complete:", output_path)
                

Download complete: Aquarium_Data.zip


- 압축파일 해제하기

In [6]:
import zipfile

with zipfile.ZipFile('./Aquarium_Data.zip') as target_file:
    target_file.extractall('./Aquarium_Data/')

## **2) yaml 파일 설정** (데이터셋 위치 알려주는 config file)
- YAML : 데이터 표현 양식의 한 종류
- 기본적으로 들여쓰기(indent)를 원칙으로하며 데이터는 Map(key-value)형식으로 작성

### 2-1) roboflow 에서 제공되는 data.yaml 파일 확인

In [ ]:
# Linux에서 실행하는 경우 
#!cat ./Aquarium_Data/data.yaml

# Windows에서 실행하는 경우
with open('./Aquarium_Data/data.yaml', 'r') as f:
    print(f.read())


train: ../train/images
val: ../valid/images
test: ../test/images

nc: 7
names: ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray']

roboflow:
  workspace: brad-dwyer
  project: aquarium-combined
  version: 2
  license: CC BY 4.0
  url: https://universe.roboflow.com/brad-dwyer/aquarium-combined/dataset/2


### 2-2) custom data에 대한 yaml 파일 만들기

In [13]:
# 파이썬에서 YAML 파일을 사용하기 위해 PyYAML 라이브러리 설치
!pip install PyYAML

In [ ]:
# yaml 파일을 학습이 가능하도록 경로 설정.
# key-value 데이터인 dict 데이터타입으로 data['train'], data['val'], data['nc'], data['names'] 에 넣어주는데,
# 가장 중요한 부분은 데이터 경로 설정임.

import yaml
# 디렉토리 정보, 클래스 이름(names), 클래스 수(nc) 지정하기
data = { 'train' : './Aquarium_Data/train/images/',
         'val'   : '... ',         # ... 코드 입력
         'test'  : './Aquarium_Data/test/images',
         'names' : ...     ,    # ... 코드 입력
         'nc' : ... }  # ... 코드 입력

# 데이터 경로와 클래스 정보를 저장하고 있는 딕셔너리 객체 data를 YOLOv8 학습에 필요한 새로운 이름으로 저장
with open('./Aquarium_Data/Aquarium_Data.yaml', 'w') as f:
  yaml.dump(data, f)

# Aquarium_Data.yaml 읽어서 화면에 출력
with open('./Aquarium_Data/Aquarium_Data.yaml', 'r') as f:
  aquarium_yaml = yaml.safe_load(f)
  display(aquarium_yaml)

In [ ]:
!cat /content/Aquarium_Data/Aquarium_Data.yaml

names:
- fish
- jellyfish
- penguin
- puffin
- shark
- starfish
- stingray
nc: 7
test: /content/Aquarium_Data/test/images
train: /content/Aquarium_Data/train/images/
val: /content/Aquarium_Data/valid/images/


## **2. yolov8 사용을 위한 패키지 설치 및 가져오기**
- https://github.com/ultralytics/ultralytics

In [ ]:
# 패키지 설치하기
# !pip install -q ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.5/756.5 kB 13.9 MB/s eta 0:00:00


In [ ]:
# 패키지 버전 확인하기
import ultralytics

ultralytics.checks()

Ultralytics YOLOv8.2.11 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.9/107.7 GB disk)


In [ ]:
import torch
torch.cuda.get_device_name(0)
torch.cuda.is_available()
print(torch.__version__)

# GPU가 사용 가능한지 확인하고, 사용 가능하면 CUDA를 사용하도록 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

2.2.1+cu121
Using device: cuda


## **3.모델 객체 선언하고 학습하기**

### 1) 모델 객체 선언하기

In [ ]:
# YOLO 라이브러리 가져오기
...    # ... code here

# 'yolov8n-seg.pt' 모델 선언하기 - 사전학습된 YOLOv8n detection model 로드하기
model = ...    # ... code here

100%|██████████| 6.23M/6.23M [00:00<00:00, 72.4MB/s]


In [ ]:
# Yolov8은 MS COCO 데이터 사전학습되어있어 MS COCO Dataset에 정의된 클래스 개수오 ㅏ종류 확인할 수 있음(0~79)
print(type(model.names), len(model.names))

print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

### 2) 모델 학습하기 (자신의 만든 yaml파일 지정)
- **`data`**: 데이터셋 설정 파일의 경로 지정- YAML 형식의 파일로, 훈련, 검증, 테스트 데이터 경로 및 클래스 수 정의
- **`epochs`**: 훈련에 사용할 총 에포크 수 지정
- **`patience`**: 모델 학습 중 개선이 없을 때 학습을 중단하기 전에 기다리는 에포크 수 지정
- **`batch`**: 각 훈련 스텝에서 처리할 이미지의 수를 지정
- **`imgsz`**: 입력 이미지의 해상도 정의(일반적으로 416, 512, 640과 같은 값이 사용)
- **`optimizer`**: 사용할 최적화 알고리즘사 지정(일반적으로 `SGD`, `Adam`, `AdamW`와 같은 옵션을 사용)
- **`lr0`**: 초기 학습률 지정
- **`momentum`**: 모멘텀 지정
- **`weight_decay`**: 가중치 감소(정규화)를 적용하는 값중 지정
- **`warmup_epochs`**: 초기 학습률을 높이기 위해 사용할 웜업 에포크 수 지정
- **`save`**: 훈련된 모델을 저장할지 여부 결정
- **`project` 및 `name`**: 모델 가중치 및 훈련 로그를 저장할 프로젝트 폴더와 하위 폴더 이름을 지정

In [ ]:
# 모델 학습하기 : model_train() -data : 자신의 만든 yaml파일 지정, epochs=10, patience=30, batch=32, imgsz=416 )

... # ... code here

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Aquarium_Data/Aquarium_Data.yaml, epochs=50, time=None, patience=30, batch=32, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 16.1MB/s]


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/Aquarium_Data/train/labels... 448 images, 1 backgrounds, 0 corrupt: 100%|██████████| 448/448 [00:00<00:00, 459.12it/s]

train: New cache created: /content/Aquarium_Data/train/labels.cache


...

                   all        127        909      0.762      0.669      0.703      0.402



50 epochs completed in 0.121 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.09s/it]


                   all        127        909      0.756      0.656      0.707       0.41
                  fish        127        459      0.836      0.634      0.754      0.393
             jellyfish        127        155      0.852      0.871      0.907      0.496
               penguin        127        104      0.616      0.615      0.634       0.29
                puffin        127         74      0.621      0.446      0.495      0.238
                 shark        127         57      0.662      0.632      0.641      0.401
              starfish        127         27      0.904      0.667      0.769      0.555
              stingray        127         33        0.8      0.727      0.746      0.495
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f4ae1515630>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
  

In [ ]:
# 커스텀 데이터로 학습하였기 때문에 클래수 수의 변경됨을 확인할 수 있음
print(type(model.names), len(model.names))

print(model.names)

<class 'dict'> 7
{0: 'fish', 1: 'jellyfish', 2: 'penguin', 3: 'puffin', 4: 'shark', 5: 'starfish', 6: 'stingray'}


### ▲ train 과정중에 loss, accuracy 등의 graph 데이터는 runs/detect/train/ 에 있는 results.csv 와 results.png 통해서 확인가능함

### 3) 테스트 이미지 데이터 생성 및 확인

In [ ]:
# 테스트 이미지
from glob import glob

test_image_list = glob('/content/Aquarium_Data/test/images/*')
test_image_list.sort()

for i in range(len(test_image_list)):
    print('i = ',i, test_image_list[i])

63
i =  0 /content/Aquarium_Data/test/images/IMG_2289_jpeg_jpg.rf.fe2a7a149e7b11f2313f5a7b30386e85.jpg
i =  1 /content/Aquarium_Data/test/images/IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125f001695.jpg
i =  2 /content/Aquarium_Data/test/images/IMG_2319_jpeg_jpg.rf.6e20bf97d17b74a8948aa48776c40454.jpg
i =  3 /content/Aquarium_Data/test/images/IMG_2347_jpeg_jpg.rf.7c71ac4b9301eb358cd4a832844dedcb.jpg
i =  4 /content/Aquarium_Data/test/images/IMG_2354_jpeg_jpg.rf.396e872c7fb0a95e911806986995ee7a.jpg
i =  5 /content/Aquarium_Data/test/images/IMG_2371_jpeg_jpg.rf.54505f60b6706da151c164188c305849.jpg
i =  6 /content/Aquarium_Data/test/images/IMG_2379_jpeg_jpg.rf.7dc3160c937072d26d4624c6c48e904d.jpg
i =  7 /content/Aquarium_Data/test/images/IMG_2380_jpeg_jpg.rf.a23809682eb1466c1136ca0f55de8fb5.jpg
i =  8 /content/Aquarium_Data/test/images/IMG_2387_jpeg_jpg.rf.09b38bacfab0922a3a6b66480f01b719.jpg
i =  9 /content/Aquarium_Data/test/images/IMG_2395_jpeg_jpg.rf.9f1503ad3b7a7c7938daed057cc4e9bc.j

## **4.객체 검출 (Inference or predict)**

In [ ]:
# 모델 예측하기 model.predict() - source 위치 지정 , save = True 지정하기
results = ...    # ... code here


image 1/63 /content/Aquarium_Data/test/images/IMG_2289_jpeg_jpg.rf.fe2a7a149e7b11f2313f5a7b30386e85.jpg: 416x320 1 penguin, 1 puffin, 94.3ms
image 2/63 /content/Aquarium_Data/test/images/IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125f001695.jpg: 416x320 12 penguins, 11.6ms
image 3/63 /content/Aquarium_Data/test/images/IMG_2319_jpeg_jpg.rf.6e20bf97d17b74a8948aa48776c40454.jpg: 416x320 6 penguins, 11.0ms
image 4/63 /content/Aquarium_Data/test/images/IMG_2347_jpeg_jpg.rf.7c71ac4b9301eb358cd4a832844dedcb.jpg: 416x320 1 penguin, 8.9ms
image 5/63 /content/Aquarium_Data/test/images/IMG_2354_jpeg_jpg.rf.396e872c7fb0a95e911806986995ee7a.jpg: 416x320 3 penguins, 8.6ms
image 6/63 /content/Aquarium_Data/test/images/IMG_2371_jpeg_jpg.rf.54505f60b6706da151c164188c305849.jpg: 416x320 2 fishs, 1 shark, 9.1ms
image 7/63 /content/Aquarium_Data/test/images/IMG_2379_jpeg_jpg.rf.7dc3160c937072d26d4624c6c48e904d.jpg: 416x320 2 fishs, 8.0ms
image 8/63 /content/Aquarium_Data/test/images/IMG_2380_jpeg_jpg.rf.

## **5. 결과 확인하기**

In [ ]:
print(type(results), len(results))

<class 'list'> 63


In [ ]:
import numpy as np

for result in results:
    uniq, cnt = np.unique(result.boxes.cls.cpu().numpy(), return_counts=True)  # Torch.Tensor -> numpy
    uniq_cnt_dict = dict(zip(uniq, cnt))

    print('\n{class num : counts} =', uniq_cnt_dict,'\n')

    for i, c in enumerate(result.boxes.cls):
        class_id = int(c)
        class_name = result.names[class_id]
        confidence_score = result.boxes.conf[i]  # 예측 확률
        print(f'class num: {class_id:>2} , class name: {class_name :<13}, confidence: {confidence_score:.2f}')


{class num : counts} = {2.0: 1, 3.0: 1} 

class num:  3 , class name: puffin       , confidence: 0.37
class num:  2 , class name: penguin      , confidence: 0.31

{class num : counts} = {2.0: 12} 

class num:  2 , class name: penguin      , confidence: 0.93
class num:  2 , class name: penguin      , confidence: 0.78
class num:  2 , class name: penguin      , confidence: 0.74
class num:  2 , class name: penguin      , confidence: 0.73
class num:  2 , class name: penguin      , confidence: 0.64
class num:  2 , class name: penguin      , confidence: 0.53
class num:  2 , class name: penguin      , confidence: 0.52
class num:  2 , class name: penguin      , confidence: 0.50
class num:  2 , class name: penguin      , confidence: 0.44
class num:  2 , class name: penguin      , confidence: 0.32
class num:  2 , class name: penguin      , confidence: 0.30
class num:  2 , class name: penguin      , confidence: 0.29

{class num : counts} = {2.0: 6} 

class num:  2 , class name: penguin      , con

In [ ]:
# 테스트 이미지 모두 예측 결과 이미지로 나타내기
from PIL import Image
from IPython.display import display
import os

# 이미지가 저장된 폴더 경로
image_dir = ...    # ... code here

# 폴더 내의 모든 파일을 순회
...    # ... code here



## **6. 결과 다운로드**

In [ ]:
import glob

detetced_image_list = glob.glob(('/content/runs/detect/train2/*'))
detected_image_nums = len(detetced_image_list)

print(detected_image_nums)
print(detetced_image_list)

63
['/content/runs/detect/train2/IMG_8590_MOV-5_jpg.rf.074e6d8acdd3fcad16d866c341b43769.jpg', '/content/runs/detect/train2/IMG_2582_jpeg_jpg.rf.14f175066ce74b470bf31fa0c7a096cd.jpg', '/content/runs/detect/train2/IMG_8331_jpg.rf.ec024bdf1e9de02b020b5e6505c1c58b.jpg', '/content/runs/detect/train2/IMG_2395_jpeg_jpg.rf.9f1503ad3b7a7c7938daed057cc4e9bc.jpg', '/content/runs/detect/train2/IMG_2514_jpeg_jpg.rf.6ccb3859d75fc5cfe053b1c1474254b2.jpg', '/content/runs/detect/train2/IMG_3164_jpeg_jpg.rf.06637eee0b72df791aa729807ca45c4d.jpg', '/content/runs/detect/train2/IMG_8515_jpg.rf.98a9daca7c5a5bad9872bd7fb2d4f198.jpg', '/content/runs/detect/train2/IMG_2547_jpeg_jpg.rf.9406b6f1a9fad2292c4abd28f712baaf.jpg', '/content/runs/detect/train2/IMG_2448_jpeg_jpg.rf.28ce79dab47ad525751d5407be09bc3d.jpg', '/content/runs/detect/train2/IMG_2347_jpeg_jpg.rf.7c71ac4b9301eb358cd4a832844dedcb.jpg', '/content/runs/detect/train2/IMG_8497_MOV-5_jpg.rf.3deffb208d656b7845661c5e33dd1afb.jpg', '/content/runs/detect/tra

In [ ]:
# 다운로드를 위한 inference image 압축

import zipfile
import os

if not os.path.exists('/content/detected_result/'):
    os.mkdir('/content/detected_result/')
    print('detected_result dir is created !!!')

with zipfile.ZipFile('/content/detected_result/detected_images.zip', 'w') as detected_images:

    for idx in range(detected_image_nums):
        detected_images.write(detetced_image_list[idx])

detected_result dir is created !!!


## **CVAT** 을 사용하여 Object Detection을 위한 사용자 데이터셋 정의하기
- CVAT : nnotation Platform
- https://www.cvat.ai/ : Open Data


## **7.학습된 모델 내보내기**

In [ ]:
# 모델을 내보내기
model.save('trained_yolov8n.pt')

# 내보낸 모델을 로드하여 사용
model = YOLO('trained_yolov8n.pt')

## 8. 테스트 이미지로 테스트 또는 웹캠 이용해 실시간 추론하기 - colab에서 작동안됨
- pc로 모델을 내보내고 저장된 모델을 로드하여 비디오에서 실시간 감지

In [ ]:
# 이미지에 대한 추론 수행
results = model('test_image.jpg')
print(results.pandas().xyxy[0])  # 추론 결과 출력

In [ ]:
# 학습된 모델을 로드합니다.
model = YOLO('trained_yolov8n.pt')

# 웹캠 초기화
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 모델을 사용하여 프레임에서 객체 감지
    results = model(frame)

    # 결과를 프레임에 그리기
    annotated_frame = results.render()[0]

    # 화면에 표시
    cv2.imshow("YOLOv8 Real-Time Detection", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()